In [1]:
import shorttext
import sqlite3

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
import pandas as pd

In [15]:
conn = sqlite3.connect("database.db")
df = pd.read_sql_query("select * from trainee_response;", conn)
df.tail()

,id,avatar_prompt_id,identifier,response_text,response_score,response_feedback,comment
59,60,60,18d,If you insist. They can come in.,3.33,,
60,61,61,19a,"Ok, I'm sorry. Just make sure they aren't in t...",1.33,,
61,62,62,19b,"That’ll make things difficult, but they can co...",3.00,,
62,63,63,19c,"Alright, they can come in. Make sure they don’...",1.67,,
63,64,64,19d,"Ok, if you insist. They can come in as long as...",4.33,,


## 1,3,5 round

In [26]:
def score_round(score):
    if score < 2.3:
        return 1
    elif score >= 2.3 and score < 4:
        return 3
    return 5

df['response_round_score'] = df['response_score'].apply(score_round)

## 70-30 Train test split

In [28]:
train = df.sample(frac=0.7, random_state=200)
test = df.drop(train.index)

In [124]:
train.index = range(len(train))
test.index = range(len(test))

## Pre- processing

In [125]:
from shorttext.utils import standard_text_preprocessor_1
pre = standard_text_preprocessor_1()

In [126]:
train['processed'] = train['response_text'].apply(pre)

## Making corpus

In [127]:
corpus = train['processed'].apply(lambda x : x.split(' '))

## Making DocumentTermMatrix using inverse weights

In [128]:
dtm = shorttext.utils.DocumentTermMatrix(corpus, tfidf = True)

In [ ]:
model = dtm.tr

## LDA

In [129]:
topicmodeler = shorttext.generators.LDAModeler()

In [130]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation

In [131]:
#30 topics
topicmodeler.train(train['processed'], 30)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [135]:
#Package to pre process
import gensim
from gensim.utils import simple_preprocess

#Builds a corpus of words
dictionary = gensim.corpora.Dictionary(corpus)


In [136]:
bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]

In [139]:
bow_corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1)],
 [(13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)],
 [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)],
 [(21, 1), (24, 1), (25, 1), (26, 1), (28, 1), (29, 1), (30, 1), (31, 1)],
 [(21, 1),
  (22, 1),
  (25, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1)],
 [(10, 1), (12, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)],
 [(10, 1), (21, 1), (25, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1)],
 [(51, 1), (52, 1), (53, 1), (54, 1)],
 [(17, 1), (47, 1), (55, 1), (56, 1), (57, 1), (58, 1)],
 [(21, 1),
  (22, 1),
  (25, 1),
  (39, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1)],
 [(10, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1)],
 [(71, 1), (72, 1), (73, 1), (74, 1)],
 [(10, 1),
  (43, 1),
  (49, 1),
  (67, 1),
  (75, 1),
  (76, 1),
  

In [138]:
topicmodeler.topicmodel.get_document_topics(bow_corpus[1])

IndexError: index 17 is out of bounds for axis 1 with size 17

In [91]:
#Gets the lda in a useable dataframe
lda_df = pd.DataFrame()
for i in corpus.index:
    print(topicmodeler.topicmodel.get_document_topics(bow_corpus[i]))
    #lda_df = pd.concat([lda_df, pd.DataFrame(topicmodeler.topicmodel.get_document_topics(bow_corpus[i]))[1]], axis=1)
    
lda_df = lda_df.transpose()    

IndexError: index 18 is out of bounds for axis 1 with size 17